# Make user, item, and interaction datasets
This notebook creates the official Amazon Personalize dataset resources for the
car search recommendations demo.

In [106]:
import json
import boto3
import time

cars_filename         = 'car_items.csv'
users_filename        = 'users.csv'
interactions_filename = 'interactions.csv'

dataset_group_name = 'car-dg10'

schema_version = 'v10'

bucket   = '<your-bucket>'
prefix   = 'personalize/' + schema_version

MAX_WAIT_TIME = time.time() + 60*60 # 1 hour

region      = '<region>'
account_num = '<your-account>'
role_arn    = '<your-role>'

CAR_INTERACTION_SCHEMA_NAME = 'car-interactions-schema-' + schema_version
CAR_INTERACTION_SCHEMA_ARN  = 'arn:aws:personalize:{}:{}:schema/'.format(region, account_num) + \
                                CAR_INTERACTION_SCHEMA_NAME
    
CAR_ITEM_SCHEMA_NAME = 'car-items-schema-' + schema_version
CAR_ITEM_SCHEMA_ARN  = 'arn:aws:personalize:{}:{}:schema/'.format(region, account_num) + \
                                CAR_ITEM_SCHEMA_NAME

CAR_USER_SCHEMA_NAME = 'car-users-schema-' + schema_version
CAR_USER_SCHEMA_ARN  = 'arn:aws:personalize:{}:{}:schema/'.format(region, account_num) + \
                                CAR_USER_SCHEMA_NAME

In [107]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## Create the INTERACTIONS schema
Create the INTERACTIONS schema if it is not in place already.

In [108]:
try:
    # first see if the schema is already in place
    arn = CAR_INTERACTION_SCHEMA_ARN
    response = personalize.describe_schema(schemaArn=arn)
    interactions_schema_arn = response['schema']['schemaArn']
    print(interactions_schema_arn)
except Exception as e:
    print('Schema {} did not exist, creating it...'.format(arn))
    schema = {
        "type": "record",
        "name": "Interactions",
        "namespace": "com.amazonaws.personalize.schema",
        "fields": [
            {
                "name": "USER_ID",
                "type": "string"
            },
            {
                "name": "ITEM_ID",
                "type": "string"
            },
            {
                "name": "TIMESTAMP",
                "type": "long"
            }
        ],
        "version": "1.0"
    }

    create_schema_response = personalize.create_schema(
        name   = CAR_INTERACTION_SCHEMA_NAME,
        schema = json.dumps(schema)
    )

    interactions_schema_arn = create_schema_response['schemaArn']
    print(json.dumps(create_schema_response, indent=2))

arn:aws:personalize:us-east-1:355151823911:schema/car-interactions-schema-v10


## Create the ITEMS schema
Create the ITEMS schema if it is not in place already.

In [109]:
try:
    arn = CAR_ITEM_SCHEMA_ARN
    response = personalize.describe_schema(schemaArn=arn)
    items_schema_arn = response['schema']['schemaArn']
    print(items_schema_arn)
except Exception as e:
    print('schema not found, creating new...')
    schema = {
        "type": "record",
        "name": "Items",
        "namespace": "com.amazonaws.personalize.schema",
        "fields": [
            {
                "name": "ITEM_ID",
                "type": "string"
            },
            {
                "name": "MAKE",
                "type": "string",
                "categorical": True
            },
            {
                "name": "MODEL",
                "type": "string",
                "categorical": True
            },
            {
                "name": "YEAR",
                "type": "int"
            },
            {
                "name": "MILEAGE",
                "type": "int"
            },
            {
                "name": "PRICE",
                "type": "int"
            }
            #,
#            {
#                "name": "COLOR",
#                "type": "string",
#                "categorical": True
#            }#,  Max of 5 metadata columns is the Personalize limit for now
    #        {
    #            "name": "LOCATION",
    #            "type": "string",
    #            "categorical": True
    #        }
        ],
        "version": "1.0"
    }

    create_schema_response = personalize.create_schema(
        name   = CAR_ITEM_SCHEMA_NAME,
        schema = json.dumps(schema)
    )

    items_schema_arn = create_schema_response['schemaArn']
    print(json.dumps(create_schema_response, indent=2))

arn:aws:personalize:us-east-1:355151823911:schema/car-items-schema-v10


## Create the USERS schema
Create the USERS schema if it is not in place already.

In [110]:
try:
    arn = CAR_USER_SCHEMA_ARN
    response = personalize.describe_schema(schemaArn=arn)
    users_schema_arn = response['schema']['schemaArn']
    print(users_schema_arn)
except Exception as e:
    schema = {
        "type": "record",
        "name": "Users",
        "namespace": "com.amazonaws.personalize.schema",
        "fields": [
            {
                "name": "USER_ID",
                "type": "string"
            },
            {
                "name": "AGE",
                "type": "int"
            },
            {
                "name": "GENDER",
                "type": "string",
                "categorical": True
            },
            {
                "name": "LOCATION",
                "type": "string",
                "categorical": True
            },
            {
                "name": "SALARY",
                "type": "int"
            }
        ],
        "version": "1.0"
    }

    create_schema_response = personalize.create_schema(
        name   = CAR_USER_SCHEMA_NAME,
        schema = json.dumps(schema)
    )

    users_schema_arn = create_schema_response['schemaArn']
    print(json.dumps(create_schema_response, indent=2))

arn:aws:personalize:us-east-1:355151823911:schema/car-users-schema-v10


## Create a dataset group and the datasets within it
First delete the datasets and dataset group if they already exist. Then create the dataset
group.

In [111]:
try:
    personalize.delete_dataset(datasetArn='arn:aws:personalize:{}:{}:dataset/{}/EVENT_INTERACTIONS'.format(region, account_num, dataset_group_name))
    personalize.delete_dataset(datasetArn='arn:aws:personalize:{}:{}:dataset/{}/INTERACTIONS'.format(region, account_num, dataset_group_name))
    personalize.delete_dataset(datasetArn='arn:aws:personalize:{}:{}:dataset/{}/ITEMS'.format(region, account_num, dataset_group_name))
    personalize.delete_dataset(datasetArn='arn:aws:personalize:{}:{}:dataset/{}/USERS'.format(region, account_num, dataset_group_name))
except Exception as e:
    pass

print('Waiting for datasets to be deleted...')
time.sleep(30)

try:
    personalize.delete_dataset_group(datasetGroupArn='arn:aws:personalize:{}:{}:dataset-group/{}'.format(region, account_num, dataset_group_name))
except Exception as e:
    pass
        
print('Waiting for dataset group to be deleted...')
time.sleep(30)

print('Creating new dataset group {}'.format(dataset_group_name))
create_dataset_group_response = personalize.create_dataset_group(
    name = dataset_group_name
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

Waiting for datasets to be deleted...
Waiting for dataset group to be deleted...
{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:355151823911:dataset-group/car-dg10",
  "ResponseMetadata": {
    "RequestId": "f57ad424-850b-4d8c-953b-59e99500f885",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 12 Jul 2019 05:05:06 GMT",
      "x-amzn-requestid": "f57ad424-850b-4d8c-953b-59e99500f885",
      "content-length": "87",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [112]:
max_time = time.time() + MAX_WAIT_TIME
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response['datasetGroup']['status']
    print('DatasetGroup: {}'.format(status))
    
    if status == 'ACTIVE' or status == 'CREATE FAILED':
        break
        
    time.sleep(60)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


### Create the INTERACTIONS dataset

In [113]:
dataset_type = 'INTERACTIONS'
create_dataset_response = personalize.create_dataset(
    name = 'car-interactions',
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interactions_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:355151823911:dataset/car-dg10/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "ac8eac19-0563-47a6-8976-10b5a88962d5",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 12 Jul 2019 05:06:06 GMT",
      "x-amzn-requestid": "ac8eac19-0563-47a6-8976-10b5a88962d5",
      "content-length": "89",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [114]:
print(items_schema_arn)
print(dataset_group_arn)

arn:aws:personalize:us-east-1:355151823911:schema/car-items-schema-v10
arn:aws:personalize:us-east-1:355151823911:dataset-group/car-dg10


### Create the ITEMS dataset

In [115]:
dataset_type = 'ITEMS'
create_dataset_response = personalize.create_dataset(
    name = 'car-items',
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = items_schema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:355151823911:dataset/car-dg10/ITEMS",
  "ResponseMetadata": {
    "RequestId": "c03a026b-87c0-4c64-98e9-7e47aa83b460",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 12 Jul 2019 05:06:06 GMT",
      "x-amzn-requestid": "c03a026b-87c0-4c64-98e9-7e47aa83b460",
      "content-length": "82",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Create the USERS dataset

In [116]:
dataset_type = 'USERS'
create_dataset_response = personalize.create_dataset(
    name = 'car-users',
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = users_schema_arn
)

users_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:355151823911:dataset/car-dg10/USERS",
  "ResponseMetadata": {
    "RequestId": "8ef9da8b-69a4-454f-9824-0c48ba51e032",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 12 Jul 2019 05:06:06 GMT",
      "x-amzn-requestid": "8ef9da8b-69a4-454f-9824-0c48ba51e032",
      "content-length": "82",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Create an import job for each of the datasets

In [117]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = '{}-car-interactions-import'.format(dataset_group_name),
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}/{}".format(bucket, prefix, interactions_filename)
    },
    roleArn = role_arn
)

interactions_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:355151823911:dataset-import-job/car-dg10-car-interactions-import",
  "ResponseMetadata": {
    "RequestId": "052c2e46-8eff-4bbd-aa0d-b03703f1fe8c",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 12 Jul 2019 05:06:06 GMT",
      "x-amzn-requestid": "052c2e46-8eff-4bbd-aa0d-b03703f1fe8c",
      "content-length": "120",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [118]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = '{}-car-items-import'.format(dataset_group_name),
    datasetArn = items_dataset_arn,
    dataSource = {
        'dataLocation': 's3://{}/{}/{}'.format(bucket, prefix, cars_filename)
    },
    roleArn = role_arn
)

items_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:355151823911:dataset-import-job/car-dg10-car-items-import",
  "ResponseMetadata": {
    "RequestId": "084f0e1d-f773-49e5-bf8d-1c1e9bfa81c8",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 12 Jul 2019 05:06:06 GMT",
      "x-amzn-requestid": "084f0e1d-f773-49e5-bf8d-1c1e9bfa81c8",
      "content-length": "113",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [119]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = '{}-car-users-import'.format(dataset_group_name),
    datasetArn = users_dataset_arn,
    dataSource = {
        'dataLocation': 's3://{}/{}/{}'.format(bucket, prefix, users_filename)
    },
    roleArn = role_arn
)

users_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:355151823911:dataset-import-job/car-dg10-car-users-import",
  "ResponseMetadata": {
    "RequestId": "6610e260-d247-44a5-961e-b9fc0513df63",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 12 Jul 2019 05:06:06 GMT",
      "x-amzn-requestid": "6610e260-d247-44a5-961e-b9fc0513df63",
      "content-length": "113",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Wait for the dataset import jobs to complete

In [120]:
max_time = time.time() + MAX_WAIT_TIME
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = items_dataset_import_job_arn
    )
    status = describe_dataset_import_job_response['datasetImportJob']['status']
    print('DatasetImportJob: {}'.format(status))
    
    if status == 'ACTIVE' or status == 'CREATE FAILED':
        break
        
    time.sleep(60)
    if status == 'ACTIVE':
        print('ITEMS dataset is ACTIVE.')

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


In [121]:
max_time = time.time() + MAX_WAIT_TIME
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = users_dataset_import_job_arn
    )
    status = describe_dataset_import_job_response['datasetImportJob']['status']
    print('DatasetImportJob: {}'.format(status))
    
    if status == 'ACTIVE' or status == 'CREATE FAILED':
        break
        
    time.sleep(60)
        if status == 'ACTIVE':
        print('USERS dataset is ACTIVE.')

DatasetImportJob: ACTIVE


In [123]:
max_time = time.time() + MAX_WAIT_TIME
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = interactions_dataset_import_job_arn
    )
    status = describe_dataset_import_job_response['datasetImportJob']['status']
    print('DatasetImportJob: {}'.format(status))
    
    if status == 'ACTIVE' or status == 'CREATE FAILED':
        break
        
    time.sleep(60)
    if status == 'ACTIVE':
        print('INTERACTIONS dataset is ACTIVE.')

DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
